In [19]:
from openai import OpenAI
from openai import AzureOpenAI

from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential
import os

from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery



In [20]:

load_dotenv(override=True) # take environment variables from .env.

endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
credential = AzureKeyCredential(os.environ["AZURE_SEARCH_QUERY_KEY"]) 
index_name = os.environ["AZURE_SEARCH_INDEX"]


# Set the API key and endpoint
api_key = os.getenv('AZURE_OPENAI_API_KEY')
api_endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')  # e.g., "https://<your-resource-name>.openai.azure.com/"
api_type = 'azure'
api_version = '2023-05-15'  # Use the appropriate API version

# Define the deployment name
deployment_name_chat = 'gpt-4o-global'
deployment_name_embeddings = 'text-embedding-ada-002'


In [21]:
import re 
def get_page_number(chunk_id : str): 
    page_re = r'_pages_(\d+)$'

    match = re.search(page_re, chunk_id)
    if match:
        page_number = match.group(1)
        return page_number

In [22]:
# Set query parameters for grounding the conversation on your search index
search_type="text"
use_semantic_reranker=True
sources_to_include=5

In [23]:
client = AzureOpenAI(
    azure_endpoint=api_endpoint,
    api_key=api_key,
    api_version=api_version,
)

In [24]:
search_client = SearchClient(
    endpoint, 
    index_name, 
    credential=credential)



In [25]:
 # This prompt provides instructions to the model
 GROUNDED_PROMPT="""
 You are a friendly assistant that answer questions from company's HR policy and information documents.
 Answer the query using only the sources provided below in a friendly and concise bulleted manner.
 Answer ONLY with the facts listed in the list of sources below.
 If there isn't enough information below, say you don't know.
 Do not generate answers that don't use the sources below.
 always provide the source of the information as a citation of source number in the list of sources.
 Sources:\n{sources}

 Semantic Answer:\n{semantic_response}
 """

In [26]:

 # Query is the question being asked. It's sent to the search engine and the LLM.
# user_query="what the role of a project manager?"

user_query = input("Enter your question: ")

In [27]:
#extract user search intent
SEARCH_INTENT_PROMPT = """
You are asked to read the user query, understand it, and then turn it into a search query.
The search query should be sent to the search engine to find the relevant information.
the search query should be similar to what a user would type into a search engine, and similar in meaning to the provided user query.
the user query is: '{user_query}'
return only the search query text. """

print(SEARCH_INTENT_PROMPT.format(user_query=user_query))


response = client.chat.completions.create(
    model=deployment_name_chat,
    messages=[
        {"role": "system", "content": SEARCH_INTENT_PROMPT.format(user_query=user_query)},
    ]
)

search_query = response.choices[0].message.content

print(search_query)


You are asked to read the user query, understand it, and then turn it into a search query.
The search query should be sent to the search engine to find the relevant information.
the search query should be similar to what a user would type into a search engine, and similar in meaning to the provided user query.
the user query is: 'Is pregnancy going to be covered by my health plan?'
return only the search query text. 
"Does my health insurance cover pregnancy?"


In [28]:
 # Retrieve the selected fields from the search index related to the question.
from azure.search.documents.models import (
    QueryType,
    QueryCaptionType,
    QueryAnswerType
)


vector_query = VectorizableTextQuery(text=user_query, k_nearest_neighbors=1, fields="vector", exhaustive=True)


results = search_client.search(  
    search_text=user_query,
    vector_queries=[vector_query],
    select=["parent_id", "chunk_id", "chunk", "title"],
    query_type=QueryType.SEMANTIC,
    semantic_configuration_name='my-semantic-config',
    query_caption=QueryCaptionType.EXTRACTIVE,
    query_answer=QueryAnswerType.EXTRACTIVE,
    top=sources_to_include
)



semantic_answers = results.get_answers()
semantic_response = ""
if semantic_answers:
    for answer in semantic_answers:
        if answer.highlights:
            semantic_response = answer.highlights
            # print(f"Semantic Answer: {answer.highlights}")
        else:
            semantic_response = answer.text
            # print(f"Semantic Answer: {answer.text}")
        print(f"Semantic Answer: {semantic_response}")
        print(f"Semantic Answer Score: {answer.score}\n")

In [29]:
list_of_sources = [f'\nSOURCE NO {index + 1}. {result["title"]}({get_page_number(result["chunk_id"])}): {result["chunk"]} )' for index, result in enumerate(results)]

In [30]:
print(list_of_sources)

['\nSOURCE NO 1. Northwind_Health_Plus_Benefits_Details.pdf(60): office visits, tests, and ultrasounds. Additionally, Northwind Health \n\nPlus covers any necessary vaccines or medications that may be prescribed by your doctor \n\nduring prenatal care.  \n\nDelivery and Post-natal Care \n\nNorthwind Health Plus covers all services related to the delivery of your baby, including \n\nlabor and delivery, as well as post-natal care for both mother and baby. This includes any \n\nnecessary treatments, tests, or medications prescribed by your doctor. Northwind Health \n\nPlus also covers any necessary follow-up care for both mother and baby for up to six weeks \n\npost-delivery.  \n\nExceptions \n\n\n\nNorthwind Health Plus does not cover infertility treatments or elective or cosmetic \n\nprocedures. Additionally, Northwind Health Plus does not cover any services related to the \n\ntermination of a pregnancy.  \n\nTips \n\nTo ensure you are getting the best care possible, it is important to 

In [31]:
joined_sources = "\n".join(list_of_sources)

In [32]:
print(joined_sources)


SOURCE NO 1. Northwind_Health_Plus_Benefits_Details.pdf(60): office visits, tests, and ultrasounds. Additionally, Northwind Health 

Plus covers any necessary vaccines or medications that may be prescribed by your doctor 

during prenatal care.  

Delivery and Post-natal Care 

Northwind Health Plus covers all services related to the delivery of your baby, including 

labor and delivery, as well as post-natal care for both mother and baby. This includes any 

necessary treatments, tests, or medications prescribed by your doctor. Northwind Health 

Plus also covers any necessary follow-up care for both mother and baby for up to six weeks 

post-delivery.  

Exceptions 



Northwind Health Plus does not cover infertility treatments or elective or cosmetic 

procedures. Additionally, Northwind Health Plus does not cover any services related to the 

termination of a pregnancy.  

Tips 

To ensure you are getting the best care possible, it is important to choose a doctor who is in-

netwo

In [33]:
GROUNDED_PROMPT.format(sources=joined_sources, semantic_response=semantic_response)
print(GROUNDED_PROMPT.format(sources=joined_sources, semantic_response=semantic_response))


You are a friendly assistant that answer questions from company's HR policy and information documents.
Answer the query using only the sources provided below in a friendly and concise bulleted manner.
Answer ONLY with the facts listed in the list of sources below.
If there isn't enough information below, say you don't know.
Do not generate answers that don't use the sources below.
always provide the source of the information as a citation of source number in the list of sources.
Sources:

SOURCE NO 1. Northwind_Health_Plus_Benefits_Details.pdf(60): office visits, tests, and ultrasounds. Additionally, Northwind Health 

Plus covers any necessary vaccines or medications that may be prescribed by your doctor 

during prenatal care.  

Delivery and Post-natal Care 

Northwind Health Plus covers all services related to the delivery of your baby, including 

labor and delivery, as well as post-natal care for both mother and baby. This includes any 

necessary treatments, tests, or medicatio

In [34]:
response = client.chat.completions.create(
    model=deployment_name_chat,
    messages=[
        {"role": "system", "content": GROUNDED_PROMPT.format(sources=joined_sources, semantic_response=semantic_response)},
        {"role": "user", "content": user_query}
    ]
)

In [35]:
print(user_query)
print(search_query)

Is pregnancy going to be covered by my health plan?
"Does my health insurance cover pregnancy?"


In [36]:


print(response.choices[0].message.content)


Yes, pregnancy is covered by the Northwind Health Plus plan. Here are the details:

- **Prenatal Care**: Covers all services related to prenatal care, including office visits, tests, ultrasounds, necessary vaccines, and medications prescribed by your doctor ([Source 1], [Source 2]).
- **Delivery and Post-natal Care**: Covers all services related to the delivery of your baby, including labor and delivery, necessary treatments, tests, and medications for both mother and baby, as well as any necessary follow-up care for up to six weeks post-delivery ([Source 1], [Source 2]).
- **Newborn Care**: Covers newborn care provided by a physician or other health care professional in the hospital or at an alternate birthing facility for a newborn baby up to 30 days old. This includes screening tests, physical assessments, evaluations, treatments, feeding care, and follow-up treatments and visits ([Source 4]).

Exceptions to the coverage include infertility treatments and elective or cosmetic proced